In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy as copy
import datetime
import json
from math import ceil
import multiprocessing
import logging
import operator
import os
from pathlib import Path
import random
import sys
import time
import typing
import warnings

import joblib
from joblib import delayed, Parallel
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from numpy.core.numeric import outer
import pandas as pd
from scipy.stats import mode, entropy
import seaborn as sns
import sklearn.ensemble
import sklearn.feature_selection
import sklearn.metrics
from sklearn.metrics import roc_auc_score
import sklearn.model_selection
import typer
from tqdm.auto import tqdm

root = str(Path.cwd().parents[1])
sys.path.append(root)
os.chdir(root)
import src

# Scenario A

## n_bins = 2

In [2]:
iterator = [(max_depth, fold) for max_depth in np.arange(1, 11) for fold in range(5)]

In [3]:
%%time
X, y, s = src.get_Xys()
y = y>1
ys = y.astype(str) + s.astype(int).astype(str)
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)

y_scores = list()
s_scores = list()

for train_index, test_index in tqdm(skf.split(X=X, y=ys), total=5):
    ships_train = X.iloc[train_index].index
    ships_test = X.iloc[test_index].index

    X_train, X_test = X.loc[ships_train], X.loc[ships_test]
    y_train, y_test = y.loc[ships_train], y.loc[ships_test]
    s_train, s_test = s.loc[ships_train], s.loc[ships_test]

    X_train = np.ascontiguousarray(X_train.values)
    y_train = np.ascontiguousarray(y_train.values.ravel())
    s_train = np.ascontiguousarray(s_train.values.ravel())
    X_test = np.ascontiguousarray(X_test.values)
    y_test = np.ascontiguousarray(y_test.values.ravel())
    s_test = np.ascontiguousarray(s_test.values.ravel())

    vt = sklearn.feature_selection.VarianceThreshold()
    vt.fit(X_train)
    X_train = vt.transform(X_train)
    X_test = vt.transform(X_test) 

    clf = src.FairRandomForestClassifier(
        max_depth=6, n_estimators=100, orthogonality=0, n_jobs=100, n_bins=2
    )    
    clf.fit(X_train, y_train, s_train)
    y_score = clf.predict_proba(X_test)[:,1]
    y_scores.append(sklearn.metrics.roc_auc_score(y_true=y_test, y_score=y_score))
    s_scores.append(sklearn.metrics.roc_auc_score(y_true=s_test, y_score=y_score))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 23.6 s, sys: 15.8 s, total: 39.4 s
Wall time: 5min 22s


In [4]:
print(f'y: {np.mean(y_scores):.3f} +- {np.std(y_scores):.3f}')
print(f's: {np.mean(s_scores):.3f} +- {np.std(s_scores):.3f}')

y: 0.675 +- 0.011
s: 0.660 +- 0.015


## n_bins = 10

In [5]:
iterator = [(max_depth, fold) for max_depth in np.arange(1, 11) for fold in range(5)]

In [6]:
%%time
X, y, s = src.get_Xys()
y = y>1
ys = y.astype(str) + s.astype(int).astype(str)
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)

y_scores = list()
s_scores = list()

for train_index, test_index in tqdm(skf.split(X=X, y=ys), total=5):
    ships_train = X.iloc[train_index].index
    ships_test = X.iloc[test_index].index

    X_train, X_test = X.loc[ships_train], X.loc[ships_test]
    y_train, y_test = y.loc[ships_train], y.loc[ships_test]
    s_train, s_test = s.loc[ships_train], s.loc[ships_test]

    X_train = np.ascontiguousarray(X_train.values)
    y_train = np.ascontiguousarray(y_train.values.ravel())
    s_train = np.ascontiguousarray(s_train.values.ravel())
    X_test = np.ascontiguousarray(X_test.values)
    y_test = np.ascontiguousarray(y_test.values.ravel())
    s_test = np.ascontiguousarray(s_test.values.ravel())

    vt = sklearn.feature_selection.VarianceThreshold()
    vt.fit(X_train)
    X_train = vt.transform(X_train)
    X_test = vt.transform(X_test) 

    clf = src.FairRandomForestClassifier(
        max_depth=6, n_estimators=100, orthogonality=0, n_jobs=100, n_bins=10
    )    
    clf.fit(X_train, y_train, s_train)
    y_score = clf.predict_proba(X_test)[:,1]
    y_scores.append(sklearn.metrics.roc_auc_score(y_true=y_test, y_score=y_score))
    s_scores.append(sklearn.metrics.roc_auc_score(y_true=s_test, y_score=y_score))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 25.3 s, sys: 13.7 s, total: 39 s
Wall time: 31min 39s


In [7]:
print(f'y: {np.mean(y_scores):.3f} +- {np.std(y_scores):.3f}')
print(f's: {np.mean(s_scores):.3f} +- {np.std(s_scores):.3f}')

y: 0.612 +- 0.036
s: 0.654 +- 0.017


# Scenario B

## n_bins = 2

In [2]:
iterator = [(max_depth, fold) for max_depth in np.arange(1, 11) for fold in range(5)]

In [3]:
%%time
X, y, s = src.get_Xys()
y = y>0
ys = y.astype(str) + s.astype(int).astype(str)
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)

y_scores = list()
s_scores = list()

for train_index, test_index in tqdm(skf.split(X=X, y=ys), total=5):
    ships_train = X.iloc[train_index].index
    ships_test = X.iloc[test_index].index

    X_train, X_test = X.loc[ships_train], X.loc[ships_test]
    y_train, y_test = y.loc[ships_train], y.loc[ships_test]
    s_train, s_test = s.loc[ships_train], s.loc[ships_test]

    X_train = np.ascontiguousarray(X_train.values)
    y_train = np.ascontiguousarray(y_train.values.ravel())
    s_train = np.ascontiguousarray(s_train.values.ravel())
    X_test = np.ascontiguousarray(X_test.values)
    y_test = np.ascontiguousarray(y_test.values.ravel())
    s_test = np.ascontiguousarray(s_test.values.ravel())

    vt = sklearn.feature_selection.VarianceThreshold()
    vt.fit(X_train)
    X_train = vt.transform(X_train)
    X_test = vt.transform(X_test) 

    clf = src.FairRandomForestClassifier(
        max_depth=6, n_estimators=100, orthogonality=0, n_jobs=100, n_bins=2
    )    
    clf.fit(X_train, y_train, s_train)
    y_score = clf.predict_proba(X_test)[:,1]
    y_scores.append(sklearn.metrics.roc_auc_score(y_true=y_test, y_score=y_score))
    s_scores.append(sklearn.metrics.roc_auc_score(y_true=s_test, y_score=y_score))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 17.9 s, sys: 13.2 s, total: 31.1 s
Wall time: 5min 22s


In [4]:
print(f'y: {np.mean(y_scores):.3f} +- {np.std(y_scores):.3f}')
print(f's: {np.mean(s_scores):.3f} +- {np.std(s_scores):.3f}')

y: 0.814 +- 0.003
s: 0.373 +- 0.012


## n_bins = 10

In [5]:
iterator = [(max_depth, fold) for max_depth in np.arange(1, 11) for fold in range(5)]

In [6]:
%%time
X, y, s = src.get_Xys()
y = y>0
ys = y.astype(str) + s.astype(int).astype(str)
skf = sklearn.model_selection.StratifiedKFold(shuffle=True, random_state=42)

y_scores = list()
s_scores = list()

for train_index, test_index in tqdm(skf.split(X=X, y=ys), total=5):
    ships_train = X.iloc[train_index].index
    ships_test = X.iloc[test_index].index

    X_train, X_test = X.loc[ships_train], X.loc[ships_test]
    y_train, y_test = y.loc[ships_train], y.loc[ships_test]
    s_train, s_test = s.loc[ships_train], s.loc[ships_test]

    X_train = np.ascontiguousarray(X_train.values)
    y_train = np.ascontiguousarray(y_train.values.ravel())
    s_train = np.ascontiguousarray(s_train.values.ravel())
    X_test = np.ascontiguousarray(X_test.values)
    y_test = np.ascontiguousarray(y_test.values.ravel())
    s_test = np.ascontiguousarray(s_test.values.ravel())

    vt = sklearn.feature_selection.VarianceThreshold()
    vt.fit(X_train)
    X_train = vt.transform(X_train)
    X_test = vt.transform(X_test) 

    clf = src.FairRandomForestClassifier(
        max_depth=6, n_estimators=100, orthogonality=0, n_jobs=100, n_bins=10
    )    
    clf.fit(X_train, y_train, s_train)
    y_score = clf.predict_proba(X_test)[:,1]
    y_scores.append(sklearn.metrics.roc_auc_score(y_true=y_test, y_score=y_score))
    s_scores.append(sklearn.metrics.roc_auc_score(y_true=s_test, y_score=y_score))

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 16 s, sys: 8.44 s, total: 24.5 s
Wall time: 29min 32s


In [7]:
print(f'y: {np.mean(y_scores):.3f} +- {np.std(y_scores):.3f}')
print(f's: {np.mean(s_scores):.3f} +- {np.std(s_scores):.3f}')

y: 0.841 +- 0.005
s: 0.395 +- 0.015
